In [1]:
import sys
datasetName = sys.argv[1] #'airports' # hospital, ncvoter, inspection
result_file = sys.argv[2]
train_round = sys.argv[3]

import os
import pandas as pd
import numpy as np

model_checkpoint_ = "distilbert-base-uncased"
mlm_model_dir = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-mlm')
saved_model_path = os.path.join('./saved_models/', datasetName + '-' + model_checkpoint_ + '-finetune')

model_checkpoint = mlm_model_dir #"distilbert-base-uncased"
data_prefix = '../../REEs_model_data/revision/labeled_data_400/'
rules_set_path = os.path.join(data_prefix, datasetName, datasetName + '_rules.txt')

train_pair_ids_file = os.path.join(data_prefix, datasetName, 'train_' + train_round, 'train.csv')
valid_pair_ids_file = os.path.join(data_prefix, datasetName, 'train_' + train_round, 'valid.csv')
test_pair_ids_file = os.path.join(data_prefix, datasetName, 'train_' + train_round, 'test.csv')


In [2]:
# only handle objective features
rules_set = pd.read_csv(rules_set_path)
rules_set = rules_set[['support_ratio', 'confidence', 'conciseness']].values
object_num = len(rules_set[0])

_datasets = []
for option in ('train', 'valid', 'test'):
    rule_pair_set_path = os.path.join(data_prefix, datasetName, 'train_' + train_round, option + '.csv')
    rules_pair_ids_set = pd.read_csv(rule_pair_set_path, delimiter=",")
    #rules_pair_ids_set
    # split training, validation and testing
    # prepare the rule pairs
    object_features = []
    for left_id, right_id, label in rules_pair_ids_set.values:
        obj_fea = [e1 - e2 for e1, e2 in zip(rules_set[left_id], rules_set[right_id])]
        object_features.append(obj_fea + [label])
    object_features = np.array(object_features)
    # add data
    _datasets.append(object_features)

In [3]:
train, valid, test = _datasets

train[:, 0] /= 1000.0
valid[:, 0] /= 1000.0
test[:, 0] /= 1000.0
train

array([[-1.99644971e+06,  2.23121564e-02, -6.66666667e-01,
         0.00000000e+00],
       [ 2.82000000e-01,  1.04688663e-01,  0.00000000e+00,
         0.00000000e+00],
       [ 2.57921560e+04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       ...,
       [-1.73780000e+01,  5.02704289e-02, -5.00000000e-01,
         0.00000000e+00],
       [ 8.64765902e+05, -1.80144597e-01,  0.00000000e+00,
         1.00000000e+00],
       [ 1.97217123e+06, -7.19962105e-02,  5.00000000e-01,
         1.00000000e+00]])

In [4]:

# logistic regression
from sklearn.linear_model import LogisticRegression
train_X, train_y = train[:, :-1], train[:, -1:]
clf = LogisticRegression(random_state=0).fit(train_X, train_y)

/home/user/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [5]:
from sklearn import metrics
import numpy as np
def compute_metrics(predictions, labels):
    #print(predictions, labels)
    precision = metrics.precision_score(labels, predictions)
    recall = metrics.recall_score(labels, predictions)
    f1 = metrics.f1_score(labels, predictions)
    acc = metrics.accuracy_score(labels, predictions)
    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': acc}

In [6]:
eval_X, eval_y = valid[:, :-1], valid[:, -1:]
eval_prediction = clf.predict(eval_X)
eval_prediction = np.array(eval_prediction)[:, np.newaxis]
print(compute_metrics(eval_prediction, eval_y))

{'precision': 0.45555555555555555, 'recall': 0.5694444444444444, 'f1': 0.5061728395061729, 'accuracy': 0.4968553459119497}


In [7]:
test_X, test_y = test[:, :-1], test[:, -1:]
test_prediction = clf.predict(test_X)
test_prediction = np.array(test_prediction)[:, np.newaxis]
print(compute_metrics(test_prediction, test_y))

{'precision': 0.4868421052631579, 'recall': 0.5211267605633803, 'f1': 0.5034013605442176, 'accuracy': 0.546583850931677}


In [ ]:
ll = compute_metrics(test_prediction, test_y)
f = open(result_file, 'w')
f.write(str(ll))
f.close()